# Майнор "Прикладные задачи анализа данных"
## Домашнее задание 1  до 23:59 17.02.2018
**Участники:** Кузнецов Евгений (ИАД3), Якименко Александра (ИАД3).

**Тема:** Генератор описания прогноза погоды на следующую неделю в городе Калининград.

## 1. Сбор данных [3 балла]

Выбираем город России - Калининград

Открываем для него прогноз погоды на 10 дней вперед  на сайте gismeteo.ru - https://www.gismeteo.ru/weather-kaliningrad-4225/10-days/ 

Чтобы извлечь нужные данные с сайта, будем использовать http.client чтобы получить html-страницу и библиотеку bs4 - парсер для разбора DOM-модели html/xml.

In [1]:
import http.client
from bs4 import BeautifulSoup # Для обработки HTML

In [2]:
conn = http.client.HTTPSConnection("www.gismeteo.ru") # создаем socket для подключения к gismeteo.ru
conn.request( "GET", "/weather-kaliningrad-4225/10-days/") # отправляем GET-запрос 
r = conn.getresponse() # получаем http ответ
field = r.read() # из ответа получаем html-страницу
conn.close() # закрываем соединение

Заходим на сайт и смотрим глазками на разметку таблички с погодой. Видим что нужная информация находится в классе widget_row в 0, 2, 3 и 4 рядах.
0 ряд - дата (день недели и число)
2 ряд - максимальная и минимальная температура
3 ряд - скорость ветра
4 ряд - уровень осадков

И так как месяц пишется только в 1 день или когда месяц меняется я напишу переделывание месяца в чиселки ручками (потому что по-другому как делать не доехало до меня). 

In [3]:
#предположим, что остальные месяцы на сайте называются так
d = {'янв':'01', 'фев':'02', 'мар':'03', 'апр':'04', 'май':'05', 'ин':'06', 'ил':'07', 'авг':'08', 'сен':'09', 'окт':'10', 'ноя':'11', 'дек':'12'}

In [4]:
import re # для удаления чисел из строки

soup = BeautifulSoup(field, "lxml") # создаем объект BeautifulSoup из полученного документа
rows = soup.find_all("div", class_="widget__row") # ищем в документе все элементы div с атрибутом класса widget__row
dates=[]
max_temp=[]
min_temp=[]
wind=[]
rainfall=[]

title = soup.find_all('div', class_='pageinfo_title')[0].contents[0].string

# проходим по всем полям div с атрибутом класса w_date
# находим атрибуты класса w_date__day это дни недели и атрибуты класса w_date__date это числа
# записываем в список dates, убирая пробелы и переносы строки

for date in rows[0].find_all("div", class_="w_date"):
    curr_date = (date.find("span", class_="w_date__date")).string.strip()
    # преобразуем дату в нужный вид
    curr_month = re.sub(r"\d+", "", curr_date).strip()
    if curr_month in d.keys():
        month = curr_month
        curr_date = curr_date.replace(curr_month,'').strip()
    curr_date += '.'+d[month]
    # и записываем
    dates.append( curr_date
        + " ("
        +  (date.find("div", class_="w_date__day")).string.strip().lower()
        + ")" )
        

# достаем значения максимальных температур (по полю div с атрибутом класса maxt)
# записываем в список max_temp, также убирая все лишнее
for max_t in rows[2].find_all("div", class_="maxt"):
    max_temp.append(max_t.string.strip().replace('−','-'))

# аналогично для минимальной температуры
for min_t in rows[2].find_all("div", class_="mint"):
    min_temp.append(min_t.string.strip().replace('−','-'))

# из третьего ряда записываем скорость ветра по классу w_wind__warning
for winds in rows[3].find_all("div", class_="w_wind__warning"):
    wind.append(winds.contents[1].strip())

# из четвертого извлекаем уровень осадков по классу  w_precipitation_value
for rainf in rows[4].find_all("div", class_="w_precipitation__value"):
    rainfall.append(rainf.contents[0].strip().replace(',','.'))

Проверим результаты:

In [5]:
print(title)
print(dates)
print(min_temp)
print(max_temp)
print(wind)
print(rainfall)

Погода в Калининграде на 10 дней
['17.02 (сб)', '18.02 (вс)', '19.02 (пн)', '20.02 (вт)', '21.02 (ср)', '22.02 (чт)', '23.02 (пт)', '24.02 (сб)', '25.02 (вс)', '26.02 (пн)']
['-2', '-3', '-4', '-8', '-10', '-12', '-11', '-13', '-14', '-17']
['+3', '+2', '+2', '0', '-5', '-6', '-6', '-1', '-8', '-10']
['10', '7', '7', '9', '11', '12', '13', '13', '12', '9']
['3.9', '0.9', '1.9', '0', '0.1', '0', '0', '0.6', '0', '0']


Осталось записать извлеченную информацию в таблицу
Библтотека csv для записи в формате csv и библиотека codecs для возможности использования кириллицы

In [6]:
import csv
import codecs

In [7]:
# создаем файл в кодировке utf8 и строим табличку как указано в задании
with codecs.open("weather.csv", "w", encoding= "utf8") as f:
    data = [[title] + dates,
            ["минимальная температура"] + min_temp,
            ["максимальная температура"] + max_temp,
            ["скорость ветра"] + wind,
            ["уровень осадков"] + rainfall]
    wr = csv.writer( f, quoting = csv.QUOTE_ALL) # разделяем все, что передали writer
    
    wr.writerows(data)   

In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('weather.csv',index_col=0,encoding='utf8', dtype=str)
df_style = df.style.set_caption(df.index.name).set_table_styles([
    dict(selector="caption", props=[
        ("font-size", "12pt"),
        ('font-weight', 'bold'),
        ('text-align','center')
    ])])
df.index.name = None

Получили искомую таблицу

In [10]:
df_style

,17.02 (сб),18.02 (вс),19.02 (пн),20.02 (вт),21.02 (ср),22.02 (чт),23.02 (пт),24.02 (сб),25.02 (вс),26.02 (пн)
минимальная температура,-2,-3,-4,-8,-10,-12,-11,-13,-14,-17
максимальная температура,+3,+2,+2,0,-5,-6,-6,-1,-8,-10
скорость ветра,10,7,7,9,11,12,13,13,12,9
уровень осадков,3.9,0.9,1.9,0,0.1,0,0,0.6,0,0


## 2. Генератор описания прогноза погоды [4 балла]

In [11]:
import pymorphy2
import re
morph = pymorphy2.MorphAnalyzer()

city_name = 'Калининград'

def column_name_2_day_of_the_week(h):
    short_2_full_name_map = { 
        'пн': 'понедельник', 
        'вт': 'вторник', 
        'ср': 'среда',
        'чт':'четверг',
        'пт':'пятница',
        'сб': 'суббота',
        'вс':'воскресенье'
    }
    return short_2_full_name_map[h[h.find("(")+1:h.find(")")]]

def column_name_2_date(h):
    return int(re.match('[0-9]+', h).group(0))

def with_capital(word):
    return word[:1].upper()+word[1:]

def value_with_units(val, units, suffix = '', tags=frozenset()):
    if isinstance(val, int):
        return '%i %s%s'%(val, morph.parse(units)[0].inflect(tags).make_agree_with_number(abs(val)).word, suffix)
    if isinstance(val, float):
        return '%g %s%s'%(val, morph.parse(units)[0].inflect(tags).inflect({'gent'}).word, suffix)
    pass

def change(prev, curr, threshold, units, suffix = ''):
    delta = abs(curr - prev)
    ratio = (curr - prev)/threshold
    if ratio > 2:
        return 'радикально увеличится на %s и достигнет отметки в %s'\
            %(
                value_with_units(delta, units, suffix),
                value_with_units(curr, units, suffix)
            )
    elif ratio > 1:
        return 'вырастет на %s и составит %s'\
            %(
                value_with_units(delta, units, suffix),
                value_with_units(curr, units, suffix)
            )
    elif ratio > -1:
        return 'с небольшими изменениями сохранится на уровне %s'\
            %(
                value_with_units(curr, units, suffix)
            )
    elif ratio > -2:
        return 'снизится на %s и составит %s'\
            %(
                value_with_units(delta, units, suffix),
                value_with_units(curr, units, suffix)
            )
    
    else:
        return 'значительно понизится на %s до %s'\
            %(
                value_with_units(delta, units, suffix),
                value_with_units(curr, units, suffix, tags={'gent'})
            )

t = ''

hottest = df.iloc[1].astype(float).idxmax()
hottest_temp = int(df.iloc[1][hottest])
hottest_temp_night = int(df.iloc[0][hottest])
hottest_temp_delta = hottest_temp - hottest_temp_night

t += 'Самый теплый день %s настанет в %s, %i-го числа. '\
    %(with_capital(morph.parse(city_name)[0].inflect({'gent'}).word), 
      morph.parse(column_name_2_day_of_the_week(hottest))[0].inflect({'acc2'}).word, 
      column_name_2_date(hottest)
     )
t += 'Воздух прогреется до %s тепла. '%value_with_units(hottest_temp,'градус',tags={'gent'})
t += 'Но уже вечером столбик термометра опустится на %s.\n' % value_with_units(hottest_temp_delta,'отметка')
     

coldest = df.iloc[0].astype(float).idxmin()
coldest_temp       = int(df.iloc[1][coldest])
coldest_temp_night = int(df.iloc[0][coldest])
t += 'А вот в %s, %i-го, жителям и гостям %s придется померзнуть. '\
    %(
        morph.parse(column_name_2_day_of_the_week(coldest))[0].inflect({'acc2'}).word, 
        column_name_2_date(coldest),
        with_capital(morph.parse(city_name)[0].inflect({'gent'}).word)
    )
t += 'Температура днем составит всего %s, а ночью опустится до %s.\n'\
    %(
        value_with_units(coldest_temp,'градус'),
        value_with_units(coldest_temp_night,'градус',tags={'gent'}),
    )
    
rain_days = df.columns[df.iloc[3].astype(float) > 0].tolist()
strong_wind_threshold = 10
if len(rain_days) > 0:
    
    t += 'В '+', '.join(['%s %i-го'%(
            morph.parse(column_name_2_day_of_the_week(col))[0].inflect({'acc2'}).word,
            column_name_2_date(col)
        ) for col in rain_days])
    t += ' ожидаются осадки.'
    rain_and_strong_wind_days = df.columns[(df.iloc[2].astype(float) > strong_wind_threshold) & (df.iloc[3].astype(float) > 0)].tolist()
    if len(rain_and_strong_wind_days) > 0:
        t += ' Кроме того в ' +', '.join(['%s %i-го'%(
            morph.parse(column_name_2_day_of_the_week(col))[0].inflect({'acc2'}).word,
            column_name_2_date(col)
        ) for col in rain_and_strong_wind_days])
        t += ' будет ветренно. Аккуратней, порывы ветра свыше %s с легкостью превратят ваш зонтик в параболическую антенну.'\
            %(value_with_units(strong_wind_threshold,'метров',tags={'gent'}, suffix=' в секунду'))
    t+='\n'

t += '\n\nСводка:\n'
n_columns = len(df.columns)
for i in range(1, n_columns):
    curr = df.columns[i]
    prev = df.columns[i-1]
    curr_col = df[curr]
    prev_col = df[prev]
    t+= '  в %s %i-го числа по сравнению с %s %s:\n'\
    %(
        morph.parse(column_name_2_day_of_the_week(curr))[0].inflect({'acc2'}).word,
        column_name_2_date(curr),
        morph.parse('предыдущий')[0].inflect({morph.parse(column_name_2_day_of_the_week(prev))[0].tag.gender,'ablt'}).word,
        morph.parse(column_name_2_day_of_the_week(prev))[0].inflect({'ablt'}).word,
    )
    t+='    минимальная температура %s\n' %(change(int  (prev_col[0]), int  (curr_col[0]), 2, 'градус'))
    t+='    максимальная температура %s\n'%(change(int  (prev_col[1]), int  (curr_col[1]), 2, 'градус'))
    t+='    скорость ветра %s\n'          %(change(int  (prev_col[2]), int  (curr_col[2]), 1, 'метр',' в секунду'))
    t+='    уровень осадков %s\n'         %(change(float(prev_col[3]), float(prev_col[3]), 1, 'миллиметр'))
    t+='\n'

print(t)

Самый теплый день Калининграда настанет в субботу, 17-го числа. Воздух прогреется до 3 градусов тепла. Но уже вечером столбик термометра опустится на 5 отметок.
А вот в понедельник, 26-го, жителям и гостям Калининграда придется померзнуть. Температура днем составит всего -10 градусов, а ночью опустится до -17 градусов.
В субботу 17-го, воскресенье 18-го, понедельник 19-го, среду 21-го, субботу 24-го ожидаются осадки. Кроме того в среду 21-го, субботу 24-го будет ветренно. Аккуратней, порывы ветра свыше 10 метров в секунду с легкостью превратят ваш зонтик в параболическую антенну.


Сводка:
  в воскресенье 18-го числа по сравнению с предыдущей субботой:
    минимальная температура с небольшими изменениями сохранится на уровне -3 градуса
    максимальная температура с небольшими изменениями сохранится на уровне 2 градуса
    скорость ветра значительно понизится на 3 метра в секунду до 7 метров в секунду
    уровень осадков с небольшими изменениями сохранится на уровне 3.9 миллиметра

  в

### 3. Ответьте на вопросы [3 балла]
* В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам? В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?
* Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче? 

* Генерировать текст по шаблонам может понадобиться для функционирования голосовых помощников и для составления текстового описания однотипных объектов, например, товаров. Автоматически сгенерированный текст об изменении числовых показателей нужен для составления читабельных сводок, например, для вывода результатов спортивных соревнований, информировании трейдеров о значительных изменениях на бирже за какой-срок.
* К сожалению, функционал pymorphy2 очень ограничен. Кроме того, что он может поставить слово в явно заданную форму, заявлена только возможность согласования числа с существительным, но даже она работает криво (примеры ниже), и приходится подпирать ее своими костылями. В таких условиях очень сложно как-то разнообразить использованные шаблоны, и знание синтаксической структуры тут сильно не поможет. Идеальный морфологический анализатор, на мой взгляд, должен работать на уровне словосочетаний, и уметь приводить все слова в нужную форму самостоятельно, без явного задания формы.

In [12]:
# не умеет работать с отрицательными числами, должно быть '-1 градус'
print(str(-1) + ' ' + morph.parse('градус')[0].make_agree_with_number(-5).word)

# мой вариант
print(value_with_units(-1, 'градус'))

-1 градусов
-1 градус


In [13]:
# не умеет работать с дробными числами, должно быть '1.2 градусa', читается: 'одна целая две десятых градуса'
print(str(1.2) + ' ' + morph.parse('градус')[0].make_agree_with_number(1.2).word)

# мой вариант
print(value_with_units(1.2, 'градус'))

1.2 градусов
1.2 градуса


я не нашел способа согласовать предлог '**в**' с существительным, 
например, для '**вторник**' должно получаться '**во вторник**', но для '**вазе**' должно получиться '**в вазе**' 
